In [78]:
# Working directory set
import sys
sys.path.append("../scripts/")

In [79]:
import pandas as pd
import numpy as np
import json
import seaborn as sns

from clean_data import load_and_clean_data

pd.set_option('display.max_rows', 400)

In [80]:
data = load_and_clean_data()

/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (10,17,19,23,25,27,28,29,30,32,33,36,37,38,39,40,41,42,43,50,51,83,86,89,92,94,95,97,100,103,106,110,113,116,121,124,127,130,133,136,188,190,201,212,231,232,242,244,247,249,256,262,267,273,279,280,281,282,283,284,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,305,318,320,333,334) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [81]:
data.shape

(222647, 291)

In [82]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc

X = data.drop('target', axis = 1)
y = data['target']

# 70 20 10 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .25, random_state = 42)

def run_baseline_dt(X_train, y_train, X_val, y_val):
    dt = DecisionTreeClassifier()
    dt.fit(X_train, y_train)
    y_pred = dt.predict_proba(X_val)[:,1]
    
    fpr, tpr, thresholds = roc_curve(y_val, y_pred)
    roc_auc = auc(fpr, tpr)
    
    importances = {}
    for name, importance in zip(X_val.columns, dt.feature_importances_):
        importances[name] = importance
        
    imp = pd.DataFrame.from_dict(importances, orient = 'index', columns = ['importance'])
    imp.sort_values(by = 'importance', ascending = False, inplace = True)
    
    return fpr, tpr, roc_auc, imp

In [84]:
fpr, tpr, roc_auc, imp = run_baseline_dt(X_train, y_train, X_val, y_val)

In [85]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(150286, 290)
(50096, 290)
(22265, 290)


# Feature Selection

## Drop 0 feature importance features

In [86]:
#drop columns that have 0 feature importance
imp.reset_index(inplace = True)

In [87]:
imp.drop(np.where(imp.importance == 0)[0], inplace = True)

In [88]:
cols_to_keep = np.array(imp['index'])

In [89]:
len(cols_to_keep)

258

In [90]:
X2_train = X_train[cols_to_keep]

In [91]:
X2_train.shape

(150286, 258)

In [92]:
X2_val = X_val[cols_to_keep]
X2_val.shape

(50096, 258)

## Remove Highly Correlated Features

In [93]:
from sklearn.feature_selection import mutual_info_classif

def mutual_information_ranking(X, y):
    """
    Use mutual information to rank features by importance
    """
    
    mi_scores = mutual_info_classif(X, y)

    mi_features = pd.Series(dict(zip(X.columns, mi_scores)))

    return mi_features.sort_values(ascending=False)

In [94]:
def pairwise_correlation_ranking(X, threshold):
    """
    Returns pairwise correlation of features ranked by absolute value, if above a certain threshold
    """
    
    corr = X.corr()
    
    corrdict = {}
    for i in range(len(corr)):
        for j in range(len(corr.columns)):
            if i != j and np.abs(corr.iloc[i,j] > threshold):
                corrdict[tuple(sorted([corr.columns[i], corr.columns[j]]))] = corr.iloc[i,j]
    return np.array(sorted(corrdict.items(), key=lambda x: np.abs(x[1]), reverse=True), dtype=object)

In [95]:
def filter_by_correlation(X, y, hp=None):
    """
    Greedy filtering of features by correlation
    """
    threshold = 0.85
    if hp is not None:
        threshold = hp['threshold']
    correlations = pairwise_correlation_ranking(X, threshold)
    while len(correlations) > 0:
        worst_feature = mutual_information_ranking(X[list(correlations[0][0])], y).idxmin()
        X.drop(worst_feature, axis=1, inplace=True)
        correlations = pairwise_correlation_ranking(X, threshold)
    return X

In [96]:
X3_train = filter_by_correlation(X2_train, y_train)

/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retu

In [97]:
X3_train.columns

Index(['OTHBLEED', 'DOPTODIS', 'MORBPROB', 'OTHSYSEP', 'OPTIME', 'MORTPROB',
       'PRWBC', 'PRPLATE', 'bmi', 'PRHCT',
       ...
       'COL_NODESEVAL_73', 'COL_NODESEVAL_63', 'SURGSPEC_10', 'ANESTHES_6',
       'VENTPATOS', 'COL_NODESEVAL_52', 'SURGSPEC_5', 'COL_NODESEVAL_59',
       'COL_NODESEVAL_66', 'COL_APPROACH_3'],
      dtype='object', length=250)

In [98]:
X3_val = X2_val[X3_train.columns]

In [99]:
X3_val.shape

(50096, 250)

## Recursive Feature Elimination (RFE)

In [100]:
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier

In [101]:
estimator = DecisionTreeClassifier()
num_features = 125
step = 5
rfe_dt = RFE(estimator, n_features_to_select = num_features, step = step).fit(X3_train, y_train)

In [102]:
dt_feat = X3_train.columns[rfe_dt.support_]
dt_feat

Index(['OTHBLEED', 'DOPTODIS', 'MORBPROB', 'OTHSYSEP', 'OPTIME', 'MORTPROB',
       'PRWBC', 'PRPLATE', 'bmi', 'PRHCT',
       ...
       'DISCHDEST_6', 'COL_NODESEVAL_26', 'COL_NODESEVAL_23',
       'WOUND_CLOSURE_3', 'COL_MALIGNANCYT_6', 'COL_NODESEVAL_20',
       'COL_MARGINS_2', 'COL_MALIGNANCYT_5', 'COL_MALIGNANCYN_7', 'DYSPNEA_2'],
      dtype='object', length=125)

In [103]:
estimator = XGBClassifier()
rfe_xgb = RFE(estimator, n_features_to_select = num_features, step = step).fit(X3_train, y_train)

/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:32:35] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:33:26] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[12:46:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:47:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:47:51] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [106]:
xgb_feat = X3_train.columns[rfe_xgb.support_]

#see how many features these two have in common by using set
all_feat_rfe = set().union(dt_feat, xgb_feat)
print(f"There are {len(all_feat_rfe)} features in the union of the 2 RFEs")

There are 160 features in the union of the 2 RFEs


In [107]:
rfe_feat = list(all_feat_rfe)
rfe_feat

['COL_MALIGNANCYM_2',
 'COL_NODESEVAL_13',
 'DISCHDEST_8',
 'COL_MALIGNANCYM_1',
 'COL_NODESEVAL_18',
 'ASACLAS_2',
 'COL_NODESEVAL_9',
 'COL_MECH_BOWEL_PREP_2',
 'SUPINFEC',
 'COL_MALIGNANCYT_3',
 'COL_MALIGNANCYM_4',
 'COL_NODESEVAL_15',
 'ASCITES',
 'DOPTODIS',
 'COL_NODESEVAL_5',
 'BLEEDIS_1',
 'OPRENAFL',
 'PRNCPTX_3',
 'COL_MECH_BOWEL_PREP_1',
 'PRPLATE',
 'COL_MALIGNANCYM_5',
 'COL_NODESEVAL_16',
 'COL_NODESEVAL_20',
 'OTHCDIFF_1',
 'COL_NODESEVAL_17',
 'COL_ANASTOMOTIC_2',
 'COL_NODESEVAL_8',
 'COL_ANASTOMOTIC_3',
 'OTHCDIFF_2',
 'COL_ORAL_ANTIBIOTIC_1',
 'SSSIPATOS',
 'COL_MALIGNANCYN_4',
 'COL_NODESEVAL_36',
 'HXCOPD',
 'num_concurr_procs',
 'COL_ORAL_ANTIBIOTIC_2',
 'PULEMBOL',
 'PRNCPTX_2',
 'PRSODM',
 'COL_ANASTOMOTIC_5',
 'ASACLAS_3',
 'COL_NODESEVAL_10',
 'COL_NODESEVAL_26',
 'COL_MALIGNANCYN_2',
 'ANESTHES_5',
 'REINTUB',
 'COL_NODESEVAL_2',
 'ADMQTR',
 'COL_NODESEVAL_69',
 'URNINFEC',
 'female',
 'FNSTATUS2_2',
 'UTIPATOS',
 'COL_MALIGNANCYT_2',
 'COL_ILEUS_1',
 'EMERG

In [108]:
X4_train = X3_train[rfe_feat]
X4_train.shape

(150286, 160)

In [109]:
X4_val = X2_val[rfe_feat]
X4_val.shape

(50096, 160)

In [110]:
dt = DecisionTreeClassifier()
dt.fit(X4_train, y_train)
y_pred = dt.predict_proba(X4_val)[:,1]

fpr, tpr, thresholds = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)
roc_auc

0.7128919900190069

## Select K Best

In [111]:
from sklearn.feature_selection import SelectKBest, f_classif
from collections import defaultdict

In [112]:
k_best = defaultdict(list)
k_vals = [125, 110, 100, 90, 85, 75, 65, 50]
cols = X4_train.columns
for k in k_vals:
    train = SelectKBest(f_classif, k).fit(X4_train, y_train)
    mask = train.get_support()
    columns = cols[mask]
    train = X4_train[columns]
    val = X4_val[columns]
    fpr, tpr, roc_auc, imp = run_baseline_dt(train, y_train, val, y_val)
    roc_auc = auc(fpr, tpr)
    k_best[k] = [roc_auc, columns]

/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass k=125 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass k=110 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass k=100 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass k=90 as keyword args. From version 0.25 passing these as position

In [113]:
k_best

defaultdict(list,
            {125: [0.7132282911334042,
              Index(['COL_MALIGNANCYM_2', 'DISCHDEST_8', 'COL_MALIGNANCYM_1', 'ASACLAS_2',
                     'COL_NODESEVAL_9', 'COL_MECH_BOWEL_PREP_2', 'SUPINFEC',
                     'COL_MALIGNANCYT_3', 'COL_MALIGNANCYM_4', 'ASCITES',
                     ...
                     'VENTPATOS', 'DSSIPATOS', 'COL_NODESEVAL_57', 'SMOKE', 'COL_MARGINS_2',
                     'COL_ANASTOMOTIC_4', 'WNDINFD', 'PRBUN', 'OTHSESHOCK', 'insulin'],
                    dtype='object', length=125)],
             110: [0.7063255851721969,
              Index(['COL_MALIGNANCYM_2', 'DISCHDEST_8', 'COL_MALIGNANCYM_1', 'ASACLAS_2',
                     'COL_MECH_BOWEL_PREP_2', 'SUPINFEC', 'COL_MALIGNANCYT_3',
                     'COL_MALIGNANCYM_4', 'ASCITES', 'DOPTODIS',
                     ...
                     'OUPNEUMO', 'VENTPATOS', 'DSSIPATOS', 'COL_NODESEVAL_57', 'SMOKE',
                     'COL_ANASTOMOTIC_4', 'WNDINFD', 'PRBU

In [114]:
best_cols = k_best[125][1]
X5_train = X4_train[best_cols]

In [115]:
X5_train.shape

(150286, 125)

In [116]:
k_best[125][0]

0.7132282911334042

In [117]:
k_best[100][0]

0.7105465529651891